# Setup

We take 2 EMA
EMA - 9
EMA - 15

We enter at Close of the Crossover

We shall optimise the strategy as well

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pandas_ta as ta
import sys
sys.path.append (r"C:\work\self-research\self-research\codes\codes")
import self_code as sc
import Utils as ut
import glob
import warnings
warnings.filterwarnings("ignore")

### Importing the data

In [2]:
bank_nifty = pd.read_csv(r"D:\Data\Indian\Index\Spot\BANKNIFTY.csv")

# converting timeframe to 10 min timeframe
bank_nifty = sc.resample(bank_nifty, '10min')

### Applying the indicator

pip install pandas_ta

In [3]:
bank_nifty['9ema'] = ta.ema(bank_nifty.close, length = 9)
bank_nifty['15ema'] = ta.ema(bank_nifty.close, length = 15)

In [4]:
bank_nifty.dropna(inplace = True)

---
### Backtesting 


##### Here we try to create a synthetic future whenever we get EMA crossover

In [5]:
# variables for backtesting
status = 0
entry_price = 0
entry_time = 0
row_counter = -1
pnl = pd.DataFrame(columns = ['Type', 'Entry Price', 'Entry Time', 'Exit Price', 'Exit Time', 'Points', 'Pnl'])

# starting backtesting
for row, value in bank_nifty.iterrows():
    
    # marking the current row using row counter variable
    row_counter += 1
    
    # skipping the first row for trade confirmation
    if row_counter == 0:
        continue

    # condition checking for bullish crossover
    if status == 0 and (bank_nifty.iloc[row_counter]['9ema'] > bank_nifty.iloc[row_counter]['15ema']) and (bank_nifty.iloc[row_counter - 1]['9ema'] < bank_nifty.iloc[row_counter - 1]['15ema']):
        ce = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'CE', row)
        pe = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'PE', row)
        try:
            ce_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(ce))
            pe_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(pe))

            entry_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            entry_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            entry_time = row
            status = 1

        except:
            continue
    elif status == 1 and (bank_nifty.iloc[row_counter]['9ema'] < bank_nifty.iloc[row_counter]['15ema']) and (bank_nifty.iloc[row_counter - 1]['9ema'] > bank_nifty.iloc[row_counter - 1]['15ema']):
        status = 0
        try:
            exit_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            exit_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            exit_time = row
        except:
            exit_price_ce = sc.options_simplify(ce_data).iloc[-1]['Close']
            exit_price_pe = sc.options_simplify(pe_data).iloc[-1]['Close']
            exit_time = row
        # squaring off the synthetic future
        pnl.loc[len(pnl)] = [ce, entry_price_ce, entry_time, exit_price_ce, exit_time, (exit_price_ce - entry_price_ce), (exit_price_ce - entry_price_ce) * 25]
        pnl.loc[len(pnl)] = [pe, entry_price_pe, entry_time, exit_price_pe, exit_time, (entry_price_pe - exit_price_pe), (entry_price_pe - exit_price_pe) * 25]

        ######### stop and reverse ###########
        ce = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'CE', row)
        pe = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'PE', row)
        try:
            ce_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(ce))
            pe_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(pe))

            entry_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            entry_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            entry_time = row
            status = -1

        except:
            continue


    # condition checking for bearish crossover
    elif status == 0 and (bank_nifty.iloc[row_counter]['9ema'] < bank_nifty.iloc[row_counter]['15ema']) and (bank_nifty.iloc[row_counter - 1]['9ema'] > bank_nifty.iloc[row_counter - 1]['15ema']):
        ce = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'CE', row)
        pe = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'PE', row)
        try:
            ce_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(ce))
            pe_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(pe))

            entry_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            entry_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            entry_time = row
            status = -1
            
        except:
            continue
    
    elif status == -1 and (bank_nifty.iloc[row_counter]['9ema'] > bank_nifty.iloc[row_counter]['15ema']) and (bank_nifty.iloc[row_counter - 1]['9ema'] < bank_nifty.iloc[row_counter - 1]['15ema']):
        status = 0
        try:
            exit_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            exit_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            exit_time = row
        except:
            exit_price_ce = sc.options_simplify(ce_data).iloc[-1]['Close']
            exit_price_pe = sc.options_simplify(pe_data).iloc[-1]['Close']
            exit_time = row
        # squaring off the synthetic future
        pnl.loc[len(pnl)] = [ce, entry_price_ce, entry_time, exit_price_ce, exit_time, (entry_price_ce - exit_price_ce), (entry_price_ce - exit_price_ce) * 25]
        pnl.loc[len(pnl)] = [pe, entry_price_pe, entry_time, exit_price_pe, exit_time, (exit_price_pe - entry_price_pe), (exit_price_pe - entry_price_pe) * 25]
        ce = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'CE', row)
        pe = ut.prepareWeeklyOptionsSymbol('BANKNIFTY', ut.priceToATM(value['close'], 'BANKNIFTY'), 'PE', row)
        try:
            ce_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(ce))
            pe_data = pd.read_csv(r"D:\Data\Indian\Index\Options\banknifty options\{}.csv".format(pe))

            entry_price_ce = sc.options_simplify(ce_data)[row:].iloc[9]['Close']
            entry_price_pe = sc.options_simplify(pe_data)[row:].iloc[9]['Close']
            entry_time = row
            status = 1

        except:
            continue


In [7]:
sc.final(pnl, r'C:\work\balysis\Youtube\Trading Strategies\EMA Crossover\result close.xlsx', capital = 150000)